In [1]:
from keras.preprocessing.video import video_to_array

from work.models.c3d import C3D


from work.dataset.activitynet import ActivityNetDataset
dataset = ActivityNetDataset(
    videos_path='../dataset/videos.json',
    labels_path='../dataset/labels.txt'
)
video = None
for v in dataset.videos:
    if v.video_id == 'gwZleaX_ZR8':
        video = v
print(video.serialize())
video_path = '../downloads/dataset/validation/'+video.video_id+'.mp4'
nb_instances = video.num_frames // 16

vid_array = video_to_array(video_path, start_frame=0,
   end_frame=16*nb_instances,
   resize=(112,112))
vid_array = vid_array.transpose(1, 0, 2, 3)
vid_array = vid_array.reshape((nb_instances, 16, 3, 112, 112))
vid_array = vid_array.transpose(0, 2, 1, 3, 4)

{'num_frames': 4478, 'subset': 'validation', 'url': 'https://www.youtube.com/watch?v=gwZleaX_ZR8', 'annotations': [{'label': 'Plataform diving', 'segment': [12.603327708618549, 31.04152935641235]}, {'label': 'Plataform diving', 'segment': [38.510167998556675, 49.01294108907213]}, {'label': 'Plataform diving', 'segment': [54.38102511311336, 71.65225197307213]}, {'label': 'Plataform diving', 'segment': [62.31645367039172, 64.4170082884948]}, {'label': 'Plataform diving', 'segment': [67.45114273686593, 71.65225197307213]}, {'label': 'Plataform diving', 'segment': [73.98620154874222, 77.48712591224738]}, {'label': 'Plataform diving', 'segment': [78.88749565764942, 84.02218472412365]}, {'label': 'Plataform diving', 'segment': [109.46223509892775, 121.36537793484527]}, {'label': 'Plataform diving', 'segment': [131.1679661526597, 136.30265521913392]}, {'label': 'Plataform diving', 'segment': [137.00284009183497, 142.3709241158762]}], 'resolution': '854x480', 'duration': 149.61}


In [2]:
import numpy as np
print(vid_array.shape)
np.save('vid.npy', vid_array)

(279, 3, 16, 112, 112)


In [2]:
from keras.layers.convolutional import (Convolution3D, MaxPooling3D,
                                        ZeroPadding3D)
from keras.layers.core import Dense, Dropout, Flatten
from keras.models import Sequential


model = Sequential()
# 1st layer group
model.add(Convolution3D(64, 3, 3, 3, activation='relu',
                        border_mode='same', name='conv1',
                        subsample=(1, 1, 1),
                        input_shape=(3, 16, 112, 112),
                        trainable=False))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                       border_mode='valid', name='pool1'))
# 2nd layer group
model.add(Convolution3D(128, 3, 3, 3, activation='relu',
                        border_mode='same', name='conv2',
                        subsample=(1, 1, 1),
                        trainable=False))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                       border_mode='valid', name='pool2'))
# 3rd layer group
model.add(Convolution3D(256, 3, 3, 3, activation='relu',
                        border_mode='same', name='conv3a',
                        subsample=(1, 1, 1),
                        trainable=False))
model.add(Convolution3D(256, 3, 3, 3, activation='relu',
                        border_mode='same', name='conv3b',
                        subsample=(1, 1, 1),
                        trainable=False))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                       border_mode='valid', name='pool3'))
# 4th layer group
model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                        border_mode='same', name='conv4a',
                        subsample=(1, 1, 1),
                        trainable=False))
model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                        border_mode='same', name='conv4b',
                        subsample=(1, 1, 1),
                        trainable=False))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                       border_mode='valid', name='pool4'))
# 5th layer group
model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                        border_mode='same', name='conv5a',
                        subsample=(1, 1, 1),
                        trainable=False))
model.add(Convolution3D(512, 3, 3, 3, activation='relu',
                        border_mode='same', name='conv5b',
                        subsample=(1, 1, 1),
                        trainable=False))
model.add(ZeroPadding3D(padding=(0, 1, 1), name='zeropadding'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                       border_mode='valid', name='pool5'))
model.add(Flatten(name='flatten'))
# FC layers group
model.add(Dense(4096, activation='relu', name='fc6', trainable=False))
model.add(Dropout(.5, name='do1'))
model.add(Dense(4096, activation='relu', name='fc7'))
model.add(Dropout(.5, name='do2'))
model.add(Dense(487, activation='softmax', name='fc8'))

# Load weights
model.load_weights('../models/c3d/c3d-sports1M_weights.h5')

for _ in range(4):
    model.layers.pop()
    model.layers[-1].outbound_nodes = []
    model.outputs = [model.layers[-1].output]


In [10]:
from work.models.c3d import C3D_conv_features
import numpy as np

vid_array = np.load('vid.npy')

model.compile(optimizer='sgd', loss='mse')
print('Model compiled')

features_old = model.predict(np.array([vid_array[0,...]]), batch_size=16)

Model compiled


In [5]:
features_new = np.load('../downloads/features/gwZleaX_ZR8.npy')

In [11]:
np.set_printoptions(threshold=np.nan)
print(features_new[0][:10])
print(features_old[0][:10])

[ 0.47422138  0.          0.          0.          0.58636814  0.
  1.72498202  0.          0.          0.76722485]
[  0.00000000e+00  -2.98023224e-08   2.97839546e+00   3.59892428e-01
   1.57966876e+00   1.12370563e+00   1.38492525e+00   1.91560030e-01
   2.87592471e-01   8.39466393e-01]


In [13]:
diff = features_new-features_old

In [14]:
print(diff[0])

[  4.74221379e-01   2.98023224e-08  -2.97839546e+00  -3.59892428e-01
  -9.93300617e-01  -1.12370563e+00   3.40056777e-01  -1.91560030e-01
  -2.87592471e-01  -7.22415447e-02   0.00000000e+00  -1.29172850e+00
   1.61051959e-01  -1.77914846e+00   0.00000000e+00   0.00000000e+00
   8.48496139e-01   3.74422073e-01  -3.39816034e-01   0.00000000e+00
  -2.98023224e-08   1.84823567e+00  -1.36067271e-01  -3.89976680e-01
   4.04511690e-01   6.28220439e-01  -8.03688824e-01  -7.01138318e-01
   8.90091777e-01  -4.28472340e-01  -4.31456447e-01   5.50676584e-02
   3.15510750e-01  -8.52100313e-01   0.00000000e+00   1.50639832e-01
  -1.38383746e-01  -1.39883637e-01  -5.86925864e-01   1.48521811e+00
  -1.13638639e-01  -1.46592021e-01  -6.40559137e-01   2.98023224e-08
   3.18965912e-02   0.00000000e+00  -2.53227115e-01   2.98023224e-08
   1.02078021e-01   1.44847810e-01   3.37688982e-01  -2.98023224e-08
  -7.82426357e-01   2.32432485e-01  -1.65148783e+00   2.44397521e-02
  -3.04007709e-01  -4.86695379e-01